In [11]:
import pickle
import pandas as pd
import re

# function to read data from file
def read_dataset(file_name,language):
    path="../data/"+language+"/"+file_name+".conllu"
    data = pd.read_csv (path, sep = '\t',quoting=3, names=["POSITION","WORD","TAG"])
    return data



In [4]:
# Definiamo una funzione per estrarre le entità nominate da una frase
def extract_entities(sentence):
    entities = []
    current_entity = None
    for idx, (word, tag) in enumerate(sentence):
        if tag.startswith('B-'):
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = {'type': tag[2:], 'start': idx, 'end': idx, 'words': [word]}
        elif tag.startswith('I-'):
            if current_entity is not None:
                current_entity['end'] = idx
                current_entity['words'].append(word)
        else:
            if current_entity is not None:
                entities.append(current_entity)
                current_entity = None
    if current_entity is not None:
        entities.append(current_entity)
    return entities

# Definiamo una funzione per dividere il testo in frasi
def split_into_sentences(text):
    return re.split(r'(?<=[.!?]) +', text)

# Definiamo una funzione per creare le quadruple
def create_quadruples(sentence, entities, sentence_idx):
    quadruples = []
    for entity in entities:
        quadruples.append((entity['type'], f"sent-{sentence_idx}", entity['start'], entity['end'], ' '.join(entity['words'])))
    return quadruples

# Funzione che dato in ingresso un dataframe di tipo (POSITION - WORD - TAG) 
#mi restituisce un DataFrame delle Entità espresse in quadruple
def process_dataframe(df):
    quadruples = []
    current_sentence = []
    for idx, row in df.iterrows():
        if isinstance(row['WORD'], float):
            # Convertiamo il float in una stringa
            word = str(row['WORD'])
        else:
            word = row['WORD']
            
        if word in ('.', '!', '?'):
            current_sentence.append((word, row['TAG']))
            entities = extract_entities(current_sentence)
            quadruples.extend(create_quadruples(current_sentence, entities, len(quadruples) + 1))
            current_sentence = []
        else:
            current_sentence.append((word, row['TAG']))
    quadruples_df = pd.DataFrame(quadruples, columns=['TAG', 'Contesto', 'Indice_inizio', 'Indice_fine', 'Entità'])
    return quadruples_df




In [5]:
# Definiamo una funzione per calcolare l'accuratezza dei singoli tag, vuole in ingresso i dataframe "classici" (POSITION -WORD -TAG)

def calculate_accuracy(system_df, golden_df):
    # Uniamo i due dataframe per confrontare i tag
    merged_df = pd.merge(system_df, golden_df, left_index=True, right_index=True, suffixes=('_system', '_golden'))
    
    # Contiamo quante volte i tag corrispondenti sono uguali
    correct_tags = (merged_df['TAG_system'] == merged_df['TAG_golden']).sum()
    
    # Calcoliamo l'accuratezza
    accuracy = correct_tags / len(system_df)
    
    return accuracy


In [6]:
# Funzione per il calcolo di precisione e recall delle entità, come valori di ingresso vuole i dataframe delle entità (Quadruple)

def calculate_precision_recall(system_df, golden_df):
    # Convertiamo i dataframe in insiemi di tuple per facilitare il confronto
    system_entities = set(map(tuple, system_df.values))
    golden_entities = set(map(tuple, golden_df.values))
    
    # Contiamo le entità estratte correttamente
    true_positives = len(system_entities & golden_entities)
    
    # Calcoliamo la precisione e il recall
    precision = true_positives / len(system_entities) if len(system_entities) > 0 else 0
    recall = true_positives / len(golden_entities) if len(golden_entities) > 0 else 0
    
    return precision, recall





In [17]:
Vit_df=read_dataset("viterbi_tag","it")
golden_df=Vit_df
acc=calculate_accuracy(Vit_df,golden_df)
vit_quadruples_df = process_dataframe(Vit_df)
gol_quadruples_df = process_dataframe(golden_df)
precision, recall =calculate_precision_recall(Vit_df,golden_df)

print("Accuracy:"+ format(acc))
print("precision:"+ format(precision))
print("recall:"+ format(recall))

Accuracy:1.0
precision:1.0
recall:1.0
